<a href="https://colab.research.google.com/github/pranavsrinivas29/Pyspark/blob/main/Pyspark_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=68f857aa9ff5fb5e88f9b9f3a551a0b008e54374278af4cf2c07ab7766fee2aa
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Grouping').getOrCreate()

Mounted at /content/drive


In [3]:
df_pyspark = spark.read.option('header','true').csv('/content/drive/MyDrive/Book1.csv',sep=';',inferSchema=True)
df_pyspark.show()

+------+---+------+----------+
|  Name|Age|Salary|Experience|
+------+---+------+----------+
|  John| 23| 34000|         8|
| Smith| 21| 56000|         5|
| Steve| 43| 45000|         2|
|  Mark| 25| 23990|        10|
| Starc| 35| 45000|         6|
|  Jade| 34| 23000|         2|
|Austin| 34| 12000|         5|
+------+---+------+----------+



In [4]:
df_pyspark.columns

['Name', 'Age', 'Salary', 'Experience']

In [10]:
from pyspark.ml.feature import VectorAssembler
feature = VectorAssembler(inputCols=['Age','Experience'],outputCol='Independent Feature')


In [11]:
output=feature.transform(df_pyspark)

In [12]:
output.show()

+------+---+------+----------+-------------------+
|  Name|Age|Salary|Experience|Independent Feature|
+------+---+------+----------+-------------------+
|  John| 23| 34000|         8|         [23.0,8.0]|
| Smith| 21| 56000|         5|         [21.0,5.0]|
| Steve| 43| 45000|         2|         [43.0,2.0]|
|  Mark| 25| 23990|        10|        [25.0,10.0]|
| Starc| 35| 45000|         6|         [35.0,6.0]|
|  Jade| 34| 23000|         2|         [34.0,2.0]|
|Austin| 34| 12000|         5|         [34.0,5.0]|
+------+---+------+----------+-------------------+



In [13]:
dependent_feature=output.select("Independent Feature","Salary")

In [14]:
dependent_feature.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|         [23.0,8.0]| 34000|
|         [21.0,5.0]| 56000|
|         [43.0,2.0]| 45000|
|        [25.0,10.0]| 23990|
|         [35.0,6.0]| 45000|
|         [34.0,2.0]| 23000|
|         [34.0,5.0]| 12000|
+-------------------+------+



In [31]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=dependent_feature.randomSplit([0.60,0.40])
reg=LinearRegression(featuresCol="Independent Feature",labelCol="Salary")
reg=reg.fit(train_data)

In [32]:
reg.coefficients

DenseVector([-1159.581, -4615.219])

In [33]:
reg.intercept

98885.98719103452

In [34]:
pred_res=reg.evaluate(test_data)


In [38]:
test_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|         [23.0,8.0]| 34000|
|         [34.0,2.0]| 23000|
+-------------------+------+



In [36]:
pred_res.predictions.show()

+-------------------+------+-----------------+
|Independent Feature|Salary|       prediction|
+-------------------+------+-----------------+
|         [23.0,8.0]| 34000|35293.87137663034|
|         [34.0,2.0]| 23000|50229.79385569915|
+-------------------+------+-----------------+

